In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time


## Twitch Stats Scrape

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

url = 'https://twitchtracker.com/games/138585'

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [3]:
results = soup.find_all('tr')[1:28]
print(len(results))

27


In [4]:
results


[<tr>
 <td data-order="2016-12-01">
 <span class="to-monthyear">2016-12-01</span>
 </td>
 <td><span class="to-number">67629</span></td>
 <td><span class="to-number">279</span></td>
 <td><span class="to-number">225057</span></td>
 <td><span class="to-number">694</span></td>
 <td><span class="to-number">50110767</span></td>
 </tr>, <tr>
 <td data-order="2017-01-01">
 <span class="to-monthyear">2017-01-01</span>
 </td>
 <td><span class="to-number">50758</span></td>
 <td><span class="to-number">242</span></td>
 <td><span class="to-number">100411</span></td>
 <td><span class="to-number">418</span></td>
 <td><span class="to-number">37610475</span></td>
 </tr>, <tr>
 <td data-order="2017-02-01">
 <span class="to-monthyear">2017-02-01</span>
 </td>
 <td><span class="to-number">49671</span></td>
 <td><span class="to-number">222</span></td>
 <td><span class="to-number">139945</span></td>
 <td><span class="to-number">367</span></td>
 <td><span class="to-number">33354988</span></td>
 </tr>, <tr>
 

In [5]:
twitch_stats = []

for result in results:
    # Splinter can capture a page's underlying html and use pass it to BeautifulSoup to allow us to scrape the content
    html = browser.html
    soup = bs(html, 'html.parser')
    # Error handling
    try:
        
        month = result.find_all('td')[0].text.strip()
        avg_view = result.find_all('td')[1].text
        avg_chan = result.find_all('td')[2].text
        peak_view = result.find_all('td')[3].text
        peak_chan = result.find_all('td')[4].text
        hours_watch = result.find_all('td')[5].text
        
        twitch_stats.append({"month":month, 
                             "avg_view":avg_view, 
                             "avg_chan":avg_chan, 
                             "peak_view":peak_view, 
                             "peak_chan":peak_chan, 
                             "hours_watch":hours_watch})

            
      
            
    except AttributeError as e:
        print(e)

In [6]:
twitch_stats

[{'month': '2016-12-01',
  'avg_view': '67629',
  'avg_chan': '279',
  'peak_view': '225057',
  'peak_chan': '694',
  'hours_watch': '50110767'},
 {'month': '2017-01-01',
  'avg_view': '50758',
  'avg_chan': '242',
  'peak_view': '100411',
  'peak_chan': '418',
  'hours_watch': '37610475'},
 {'month': '2017-02-01',
  'avg_view': '49671',
  'avg_chan': '222',
  'peak_view': '139945',
  'peak_chan': '367',
  'hours_watch': '33354988'},
 {'month': '2017-03-01',
  'avg_view': '52111',
  'avg_chan': '227',
  'peak_view': '241636',
  'peak_chan': '387',
  'hours_watch': '38748339'},
 {'month': '2017-04-01',
  'avg_view': '74146',
  'avg_chan': '308',
  'peak_view': '277440',
  'peak_chan': '840',
  'hours_watch': '53100504'},
 {'month': '2017-05-01',
  'avg_view': '53461',
  'avg_chan': '248',
  'peak_view': '127148',
  'peak_chan': '414',
  'hours_watch': '38755685'},
 {'month': '2017-06-01',
  'avg_view': '39147',
  'avg_chan': '203',
  'peak_view': '86314',
  'peak_chan': '335',
  'hours_

In [7]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x0000000008C02C50>>

## Tempo Storm Scrape

In [6]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

url = 'https://tempostorm.com/hearthstone/meta-snapshot/standard/2019-02-18'

browser.visit(url)


In [7]:
time.sleep(2)
html = browser.html
soup = bs(html, 'html.parser')


In [8]:
results = soup.find_all('h3', class_='ng-binding')


In [9]:
tier1 = results[10].text.replace('1. ','')
tier1

'Midrange Hunter'

In [10]:
browser.url


'https://tempostorm.com/hearthstone/meta-snapshot/standard/2019-02-18'

In [11]:
tier_one_decks = []

for x in range(3):   
    time.sleep(2)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    results = soup.find_all('h3', class_='ng-binding')
    tier1 = results[10].text.replace('1. ','')
    
    if tier1 == 'Tier 1':
        tier1 = results[11].text.replace('1. ','')
        
    print(browser.url)
    base_link = browser.url
    print(tier1)
    links_found = soup.find_all('a', class_='big-view-deck-link pull-left ng-binding')
    link = "https://tempostorm.com" + links_found[0]['href']
    
    browser.visit(link)
    print(browser.url)
    time.sleep(2)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    
    results2 = soup.find_all('div', class_='db-deck-card-name ng-binding')
    results2 = set(results2)
    deck_list = []
    for result in results2:
        deck_list.append(result.text)
    
    browser.visit(base_link)
    tier_one_decks.append({"deck":tier1,"deck_list":deck_list ,"date":browser.url.replace('https://tempostorm.com/hearthstone/meta-snapshot/standard/','')})

    time.sleep(2)
    button = browser.find_by_css('.btn-pagination-arrow.m-r-xs')    
    button.click()



https://tempostorm.com/hearthstone/meta-snapshot/standard/2019-02-18
Midrange Hunter
https://tempostorm.com/hearthstone/decks/midrange-hunter-standard-meta-snapshot-feb-18-2019
https://tempostorm.com/hearthstone/meta-snapshot/standard/2019-02-12
Midrange Hunter
https://tempostorm.com/hearthstone/decks/midrange-hunter-standard-meta-snapshot-feb-12-2019
https://tempostorm.com/hearthstone/meta-snapshot/standard/2019-01-27
Subject 9 Hunter
https://tempostorm.com/hearthstone/decks/subject-9-hunter-standard-meta-snapshot-jan-27-2019


In [12]:
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x0000000008BBEF28>>

In [13]:
tier_one_decks

[{'deck': 'Midrange Hunter',
  'deck_list': ['Dire Mole',
   'Animal Companion',
   "Master's Call",
   'Lifedrinker',
   'Tracking',
   'Crackling Razormaw',
   'Unleash the Hounds',
   'Flanking Strike',
   'Tundra Rhino',
   'Deathstalker Rexxar',
   'Scavenging Hyena',
   'Dire Frenzy',
   'Kill Command',
   'Springpaw',
   "Headhunter's Hatchet",
   'Timber Wolf'],
  'date': '2019-02-18'},
 {'deck': 'Midrange Hunter',
  'deck_list': ['Dire Mole',
   'Animal Companion',
   "Master's Call",
   'Tracking',
   'Crackling Razormaw',
   'Unleash the Hounds',
   'Flanking Strike',
   'Revenge of the Wild',
   'Deathstalker Rexxar',
   'Candleshot',
   'Scavenging Hyena',
   'Dire Frenzy',
   'Kill Command',
   'Springpaw',
   'Tundra Rhino',
   'Timber Wolf'],
  'date': '2019-02-12'},
 {'deck': 'Subject 9 Hunter',
  'deck_list': ['Animal Companion',
   'Flanking Strike',
   'Explosive Trap',
   'Candleshot',
   'Scavenging Hyena',
   "Zul'jin",
   'Lesser Emerald Spellstone',
   'Snake T

## Hearthstone JSON API

In [14]:
response =  requests.get("https://api.hearthstonejson.com/v1/28855/enUS/cards.collectible.json")

In [15]:
print(response.status_code)

200


In [19]:
card_data = response.json()

In [20]:
card_data[0]

{'artist': 'Nutthapon Petchthai',
 'cardClass': 'MAGE',
 'collectible': True,
 'cost': 5,
 'dbfId': 2539,
 'flavor': "It's on the rack next to ice lance, acid lance, and English muffin lance.",
 'id': 'AT_001',
 'name': 'Flame Lance',
 'playRequirements': {'REQ_MINION_TARGET': 0, 'REQ_TARGET_TO_PLAY': 0},
 'rarity': 'COMMON',
 'set': 'TGT',
 'text': 'Deal $8 damage to a minion.',
 'type': 'SPELL'}

In [25]:
## snippet to get specific card out of the json
list(filter(lambda card: card['name']=='Timber Wolf',card_data))

[{'artist': 'Malcolm Davis',
  'attack': 1,
  'cardClass': 'HUNTER',
  'collectible': True,
  'cost': 1,
  'dbfId': 606,
  'flavor': 'Other beasts totally dig hanging out with timber wolves.',
  'health': 1,
  'howToEarn': 'Unlocked at Level 1.',
  'howToEarnGolden': 'Unlocked at Level 23.',
  'id': 'DS1_175',
  'mechanics': ['AURA'],
  'name': 'Timber Wolf',
  'race': 'BEAST',
  'rarity': 'FREE',
  'set': 'CORE',
  'text': 'Your other Beasts have +1\xa0Attack.',
  'type': 'MINION'}]

In [28]:
tier_one_decks[0]['deck']

'Midrange Hunter'

In [31]:
len(tier_one_decks)

3

In [35]:
tier_one_decks[0]['deck']

'Midrange Hunter'

In [37]:
tier_one_decks[0]["deck_list"]

['Dire Mole',
 'Animal Companion',
 "Master's Call",
 'Lifedrinker',
 'Tracking',
 'Crackling Razormaw',
 'Unleash the Hounds',
 'Flanking Strike',
 'Tundra Rhino',
 'Deathstalker Rexxar',
 'Scavenging Hyena',
 'Dire Frenzy',
 'Kill Command',
 'Springpaw',
 "Headhunter's Hatchet",
 'Timber Wolf']

In [38]:
decks_with_cards = []
for x in range(len(tier_one_decks)):
    
    deck = tier_one_decks[x]['deck']
    
    deck_builder = []
    for card_item in tier_one_decks[x]["deck_list"]:
        deck_builder.append(list(filter(lambda card: card['name']==card_item,card_data)))
        
    decks_with_cards.append({"deck":deck,
                            "card_info":deck_builder})    
    

In [48]:
decks_with_cards[2]['card_info'][0]

[{'artist': 'Wei Wang',
  'cardClass': 'HUNTER',
  'collectible': True,
  'cost': 3,
  'dbfId': 437,
  'entourage': ['NEW1_032', 'NEW1_033', 'NEW1_034'],
  'flavor': "You could summon Misha, Leokk, or Huffer!  Huffer is more trouble than he's worth.",
  'howToEarn': 'Unlocked at Level 2.',
  'howToEarnGolden': 'Unlocked at Level 45.',
  'id': 'NEW1_031',
  'name': 'Animal Companion',
  'playRequirements': {'REQ_NUM_MINION_SLOTS': 1},
  'rarity': 'FREE',
  'set': 'CORE',
  'text': 'Summon a random Beast Companion.',
  'type': 'SPELL'}]

In [49]:
decks_with_cards

[{'deck': 'Midrange Hunter',
  'card_info': [[{'artist': 'Daren Bader',
     'attack': 1,
     'cardClass': 'NEUTRAL',
     'collectible': True,
     'cost': 1,
     'dbfId': 45707,
     'flavor': "You especially don't want to make a dire mountain out of a dire mole hill.",
     'health': 3,
     'id': 'LOOT_258',
     'name': 'Dire Mole',
     'race': 'BEAST',
     'rarity': 'COMMON',
     'set': 'LOOTAPALOOZA',
     'type': 'MINION'}],
   [{'artist': 'Wei Wang',
     'cardClass': 'HUNTER',
     'collectible': True,
     'cost': 3,
     'dbfId': 437,
     'entourage': ['NEW1_032', 'NEW1_033', 'NEW1_034'],
     'flavor': "You could summon Misha, Leokk, or Huffer!  Huffer is more trouble than he's worth.",
     'howToEarn': 'Unlocked at Level 2.',
     'howToEarnGolden': 'Unlocked at Level 45.',
     'id': 'NEW1_031',
     'name': 'Animal Companion',
     'playRequirements': {'REQ_NUM_MINION_SLOTS': 1},
     'rarity': 'FREE',
     'set': 'CORE',
     'text': 'Summon a random Beast Compa